In [1]:
import src.requestcompletion as rc
from src.requestcompletion.llm import MessageHistory, SystemMessage, UserMessage
from src.requestcompletion.nodes.library import from_function
from src.requestcompletion.visuals.agent_viewer import AgentViewer
from pydantic import BaseModel

# Image Transcription Agent Demo

We will make an image to notion agent that can look at an image and understand it.

We will then wrap these 2 Agents in a top level agent that can call them both sequencially.

### Step 1: Creating the ImageAgent

In [2]:
# Importing all functions that the notion_agent can invoke
from demo.sample_tools.image_tools import parse_image_to_text, parse_image_to_mermaid, save_image, analyze_image
parse_image_to_mermaid_node = from_function(parse_image_to_mermaid)
parse_image_to_text_node = from_function(parse_image_to_text)
save_image_node = from_function(save_image)
analyze_image_node = from_function(analyze_image)

We have already created the NotionAgent, ImageAgent, and AudioAgent in the sample_agents folder. Instead of importing and calling them directly, we will make the ImageAgent to demonstrate how easy it is to use the RC library.

In [3]:
class ImageTranscription(BaseModel):
    text: str
    code: str | None = None

SYSTEM_PROMPT = SystemMessage(
    """
    You are a helpful assistant that can parse images(mermaid code and text), analyze images, and transcribe them.
    """
)

ImageAgent = rc.library.tool_call_llm(
    pretty_name="Image Agent",
    system_message=SYSTEM_PROMPT,
    model=rc.llm.OpenAILLM(model_name="gpt-4o"),
    connected_nodes=[
        from_function(parse_image_to_mermaid),
        from_function(parse_image_to_text),
        from_function(analyze_image),
        from_function(save_image),
    ],
    output_model=ImageTranscription,
)

Yes, that is all the code you need to create a tool_calling ImageAgent 😙

### Step 2: Importing the NotionAgent

In [4]:
from demo.sample_agents import NotionAgent   

### Step 3: Wrap both agents in a runner function and create a node from the function

In [5]:
async def top_level_node(user_prompt: str):
    transcription = await rc.call(ImageAgent, 
                           message_history=MessageHistory([UserMessage(user_prompt)]))
    
    await rc.call(NotionAgent, message_history=MessageHistory([UserMessage(user_prompt),
                                                               UserMessage(f"Transctiption text: {transcription.text}"),
                                                               UserMessage(f"Transctiption code: {transcription.code}"),
                                                               ]))
    
ImageNotionAgent = from_function(top_level_node)

### Step 4: Invoke the top level node

In [6]:
USER_PROMPT = "I have a sample flowchart I made on a whiteboard, the path to the image is 'demo/assets/mermaid.jpeg'. " \
"I want you add a mermaid diagram under the 'Agent Demo Root' page in Notion."

In [7]:
with rc.Runner(executor_config=rc.ExecutorConfig(timeout=50)) as runner:
    result = await runner.run(ImageNotionAgent, 
                              user_prompt=USER_PROMPT,
                              )

[+2.872  s] RC.RUNNER   : INFO     - START CREATED top_level_node Node - (, user_prompt=I have a sample flowchart I made on a whiteboard, the path to the image is 'demo/assets/mermaid.jpeg'. I want you add a mermaid diagram under the 'Agent Demo Root' page in Notion.)
[+2.874  s] RC.RUNNER   : INFO     - top_level_node Node CREATED Image Agent - (, message_history=user: I have a sample flowchart I made on a whiteboard, the path to the image is 'demo/assets/mermaid.jpeg'. I want you add a mermaid diagram under the 'Agent Demo Root' page in Notion.)
[+4.144  s] RC.RUNNER   : INFO     - Image Agent CREATED parse_image_to_mermaid Node - ({'image_source': 'demo/assets/mermaid.jpeg'}, )


Loading image from demo/assets/mermaid.jpeg
Image successfully uploaded and encoded
Parsing image to Mermaid code...


[+9.378  s] RC.RUNNER   : INFO     - parse_image_to_mermaid Node DONE graph TD
    AudioFile[Audio File] -.-> StartupNotes[Startup Notes]
    StartupNotes --> RC[RC]
    RC --> MeetingSummary[Meeting Summary]
    RC --> MeetingStats[Meeting Stats]
    RC --> MeetingWhiteboard[Meeting Whiteboard]
    MeetingSummary --> NotionWriter[Notion Writer Tool]
    MeetingStats --> NotionWriter
    MeetingWhiteboard --> NotionWriter
    NotionWriter --> NotionAPI[Notion API]


Mermaid code generation complete


[+14.525 s] RC.RUNNER   : INFO     - Image Agent CREATED ImageTranscription - (, message_history=user: Here's the Mermaid diagram code extracted from your flowchart:

```mermaid
graph TD
    AudioFile[Audio File] -.-> StartupNotes[Startup Notes]
    StartupNotes --> RC[RC]
    RC --> MeetingSummary[Meeting Summary]
    RC --> MeetingStats[Meeting Stats]
    RC --> MeetingWhiteboard[Meeting Whiteboard]
    MeetingSummary --> NotionWriter[Notion Writer Tool]
    MeetingStats --> NotionWriter
    MeetingWhiteboard --> NotionWriter
    NotionWriter --> NotionAPI[Notion API]
```

To add this diagram to the 'Agent Demo Root' page in Notion, you can follow these steps:

1. Open the 'Agent Demo Root' page in Notion.
2. Click on the area where you want to add the diagram.
3. Type `/code` to insert a code block.
4. Paste the Mermaid code above into the code block.
5. Ensure the code block is set to render as a Mermaid diagram.

If you need further assistance with Notion, feel free to ask!)
[+18.

### Additional: Graphical representation of the nodes and the flow of data

In [8]:
viewer = AgentViewer(
        stamps=result.all_stamps,
        request_heap=result.request_heap,
        node_heap=result.node_heap
    )
viewer.display_graph()